### Wide and Deep Network Architectures

### Preparation

In [37]:
import pandas as pd
df = pd.read_csv('data.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance	                     4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Nacionality                                     4424 non-null   int64  
 8   Mother's qualification                          4424 non-null   int64  
 9   Father's qualification                   

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


Reading in the raw data, we can see that there are 7 features of type float64, 29 feature of type int64, and 1 feature of type object. We will need to convert the object feature into a numerical representation using one-hot encoding. We will also need to remove any features that are not useful for this project.

In [38]:
from sklearn.preprocessing import OneHotEncoder

df['Student_Status'] = df['Target'].map({'Dropout': 0, 'Graduate': 1, 'Enrolled': 2}).fillna(-1)
#remove unwanted columns
drop_columns = ['Application mode', 'Application order','Curricular units 1st sem (credited)', 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 'Curricular units 1st sem (without evaluations)', 'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (without evaluations)', 'Age at enrollment']
df = df.drop(drop_columns, axis=1)
# Combine the last 3 features into a single 'Student_Status' column
df['Student_Status'] = df['Target'].map({'Dropout': 0, 'Graduate': 1, 'Enrolled': 2}).fillna(-1)
df = df.drop('Target', axis=1)
# categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Change these values to floats so they're counted with the other numerical values
df['Curricular units 1st sem (enrolled)'] = df['Curricular units 1st sem (enrolled)'].astype(float)
df['Curricular units 2nd sem (enrolled)'] = df['Curricular units 2nd sem (enrolled)'].astype(float)

attr_des = pd.DataFrame()
attr_des['Features'] = df.columns
# Description taken directly from dataset website https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success
attr_des['Description'] = [ # Marital status
                            '1 – single 2 – married 3 – widower 4 – divorced 5 – facto union 6 – legally separated',
                           # Courses
                           '33 - Biofuel Production Technologies 171 - Animation and Multimedia Design 8014 - Social Service (evening attendance) 9003 - Agronomy 9070 - Communication Design 9085 - Veterinary Nursing 9119 - Informatics Engineering 9130 - Equinculture 9147 - Management 9238 - Social Service 9254 - Tourism 9500 - Nursing 9556 - Oral Hygiene 9670 - Advertising and Marketing Management 9773 - Journalism and Communication 9853 - Basic Education 9991 - Management (evening attendance)',
                            # Daytime/evening attendance
                            '1 - daytime 0 - evening',
                            # Previous qualification
                            '1 - Secondary education 2 - Higher education - bachelors degree 3 - Higher education - degree 4 - Higher education - masters 5 - Higher education - doctorate 6 - Frequency of higher education 9 - 12th year of schooling - not completed 10 - 11th year of schooling - not completed 12 - Other - 11th year of schooling 14 - 10th year of schooling 15 - 10th year of schooling - not completed 19 - Basic education 3rd cycle (9th/10th/11th year) or equiv. 38 - Basic education 2nd cycle (6th/7th/8th year) or equiv. 39 - Technological specialization course 40 - Higher education - degree (1st cycle) 42 - Professional higher technical course 43 - Higher education - master (2nd cycle)',
                            # Previous qualification (grade)
                            'Grade of previous qualification (between 0 and 200)',
                            # Nacionality
                            '1 - Portuguese; 2 - German; 6 - Spanish; 11 - Italian; 13 - Dutch; 14 - English; 17 - Lithuanian; 21 - Angolan; 22 - Cape Verdean; 24 - Guinean; 25 - Mozambican; 26 - Santomean; 32 - Turkish; 41 - Brazilian; 62 - Romanian; 100 - Moldova (Republic of); 101 - Mexican; 103 - Ukrainian; 105 - Russian; 108 - Cuban; 109 - Colombian',
                            # Mother's qualification
                            '1 - Secondary Education - 12th Year of Schooling or Eq. 2 - Higher Education - Bachelor''s Degree 3 - Higher Education - Degree 4 - Higher Education - Master''s 5 - Higher Education - Doctorate 6 - Frequency of Higher Education 9 - 12th Year of Schooling - Not Completed 10 - 11th Year of Schooling - Not Completed 11 - 7th Year (Old) 12 - Other - 11th Year of Schooling 14 - 10th Year of Schooling 18 - General commerce course 19 - Basic Education 3rd Cycle (9th/10th/11th Year) or Equiv. 22 - Technical-professional course 26 - 7th year of schooling 27 - 2nd cycle of the general high school course 29 - 9th Year of Schooling - Not Completed 30 - 8th year of schooling 34 - Unknown 35 - Can''t read or write 36 - Can read without having a 4th year of schooling 37 - Basic education 1st cycle (4th/5th year) or equiv. 38 - Basic Education 2nd Cycle (6th/7th/8th Year) or Equiv. 39 - Technological specialization course 40 - Higher education - degree (1st cycle) 41 - Specialized higher studies course 42 - Professional higher technical course 43 - Higher Education - Master (2nd cycle) 44 - Higher Education - Doctorate (3rd cycle)',
                            # Father's qualification
                            '1 - Secondary Education - 12th Year of Schooling or Eq. 2 - Higher Education - Bachelor''s Degree 3 - Higher Education - Degree 4 - Higher Education - Master''s 5 - Higher Education - Doctorate 6 - Frequency of Higher Education 9 - 12th Year of Schooling - Not Completed 10 - 11th Year of Schooling - Not Completed 11 - 7th Year (Old) 12 - Other - 11th Year of Schooling 13 - 2nd year complementary high school course 14 - 10th Year of Schooling 18 - General commerce course 19 - Basic Education 3rd Cycle (9th/10th/11th Year) or Equiv. 20 - Complementary High School Course 22 - Technical-professional course 25 - Complementary High School Course - not concluded 26 - 7th year of schooling 27 - 2nd cycle of the general high school course 29 - 9th Year of Schooling - Not Completed 30 - 8th year of schooling 31 - General Course of Administration and Commerce 33 - Supplementary Accounting and Administration 34 - Unknown 35 - Can''t read or write 36 - Can read without having a 4th year of schooling 37 - Basic education 1st cycle (4th/5th year) or equiv. 38 - Basic Education 2nd Cycle (6th/7th/8th Year) or Equiv. 39 - Technological specialization course 40 - Higher education - degree (1st cycle) 41 - Specialized higher studies course 42 - Professional higher technical course 43 - Higher Education - Master (2nd cycle) 44 - Higher Education - Doctorate (3rd cycle)',
                            # Mother's occupation
                            '0 - Student 1 - Representatives of the Legislative Power and Executive Bodies, Directors, Directors and Executive Managers 2 - Specialists in Intellectual and Scientific Activities 3 - Intermediate Level Technicians and Professions 4 - Administrative staff 5 - Personal Services, Security and Safety Workers and Sellers 6 - Farmers and Skilled Workers in Agriculture, Fisheries and Forestry 7 - Skilled Workers in Industry, Construction and Craftsmen 8 - Installation and Machine Operators and Assembly Workers 9 - Unskilled Workers 10 - Armed Forces Professions 90 - Other Situation 99 - (blank) 122 - Health professionals 123 - teachers 125 - Specialists in information and communication technologies (ICT) 131 - Intermediate level science and engineering technicians and professions 132 - Technicians and professionals, of intermediate level of health 134 - Intermediate level technicians from legal, social, sports, cultural and similar services 141 - Office workers, secretaries in general and data processing operators 143 - Data, accounting, statistical, financial services and registry-related operators 144 - Other administrative support staff 151 - personal service workers 152 - sellers 153 - Personal care workers and the like 171 - Skilled construction workers and the like, except electricians 173 - Skilled workers in printing, precision instrument manufacturing, jewelers, artisans and the like 175 - Workers in food processing, woodworking, clothing and other industries and crafts 191 - cleaning workers 192 - Unskilled workers in agriculture, animal production, fisheries and forestry 193 - Unskilled workers in extractive industry, construction, manufacturing and transport 194 - Meal preparation assistants',
                            # Father's occupation
                            '0 - Student 1 - Representatives of the Legislative Power and Executive Bodies, Directors, Directors and Executive Managers 2 - Specialists in Intellectual and Scientific Activities 3 - Intermediate Level Technicians and Professions 4 - Administrative staff 5 - Personal Services, Security and Safety Workers and Sellers 6 - Farmers and Skilled Workers in Agriculture, Fisheries and Forestry 7 - Skilled Workers in Industry, Construction and Craftsmen 8 - Installation and Machine Operators and Assembly Workers 9 - Unskilled Workers 10 - Armed Forces Professions 90 - Other Situation 99 - (blank) 101 - Armed Forces Officers 102 - Armed Forces Sergeants 103 - Other Armed Forces personnel 112 - Directors of administrative and commercial services 114 - Hotel, catering, trade and other services directors 121 - Specialists in the physical sciences, mathematics, engineering and related techniques 122 - Health professionals 123 - teachers 124 - Specialists in finance, accounting, administrative organization, public and commercial relations 131 - Intermediate level science and engineering technicians and professions 132 - Technicians and professionals, of intermediate level of health 134 - Intermediate level technicians from legal, social, sports, cultural and similar services 135 - Information and communication technology technicians 141 - Office workers, secretaries in general and data processing operators 143 - Data, accounting, statistical, financial services and registry-related operators 144 - Other administrative support staff 151 - personal service workers 152 - sellers 153 - Personal care workers and the like 154 - Protection and security services personnel 161 - Market-oriented farmers and skilled agricultural and animal production workers 163 - Farmers, livestock keepers, fishermen, hunters and gatherers, subsistence 171 - Skilled construction workers and the like, except electricians 172 - Skilled workers in metallurgy, metalworking and similar 174 - Skilled workers in electricity and electronics 175 - Workers in food processing, woodworking, clothing and other industries and crafts 181 - Fixed plant and machine operators 182 - assembly workers 183 - Vehicle drivers and mobile equipment operators 192 - Unskilled workers in agriculture, animal production, fisheries and forestry 193 - Unskilled workers in extractive industry, construction, manufacturing and transport 194 - Meal preparation assistants 195 - Street vendors (except food) and street service providers',
                            # Admission grade
                            'Admission grade (between 0 and 200)',
                            # Displaced
                            '1 – yes 0 – no',
                            # Educational special needs
                            '1 – yes 0 – no',
                            # Debtor
                            '1 – yes 0 – no',
                            # Tuition fees up to date
                            '1 - yes 0 - no',
                            # Gender
                            '1 – male 0 – female',
                            # Scholarship holder
                            '1 – yes 0 – no',
                            # International
                            '1 – yes 0 – no',
                            # Curricular units 1st sem (enrolled)
                            'Number of curricular units enrolled in the 1st semester',
                            # Curricular units 1st sem (grade)
                            'Grade average in the 1st semester (between 0 and 20)',
                            # Curricular units 2nd sem (enrolled)
                            'Number of curricular units enrolled in the 2nd semester',
                            # Curricular units 2nd sem (grade)
                            'Grade average in the 2nd semester (between 0 and 20)',
                            # Unemployment rate
                            'Unemployment rate (%)',
                            # Inflation rate
                            'Inflation rate (%)',
                            # GDP
                            'GDP',
                            # Target (Student_Status)
                            '0 - Dropout 1 - Graduate 3 - Enrolled',
                            ]
attr_des['Type'] = ['Nominal','Nominal','Nominal','Nominal','Numeric','Nominal','Nominal','Nominal','Nominal','Nominal','Numeric','Nominal','Nominal','Nominal','Nominal','Nominal','Nominal','Nominal', 'Numeric', 'Numeric', 'Numeric','Numeric','Numeric','Numeric','Numeric', 'Nominal']
attr_des['Continous/Discrete'] = ['Discrete','Discrete','Discrete','Discrete','Continous','Discrete','Discrete','Discrete','Discrete','Discrete','Continous','Discrete','Discrete','Discrete','Discrete','Discrete','Discrete','Discrete','Continous','Continous','Continous','Continous','Continous','Continous','Continous','Discrete']


attr_des

,Features,Description,Type,Continous/Discrete
0,Marital status,1 – single 2 – married 3 – widower 4 – divorce...,Nominal,Discrete
1,Course,33 - Biofuel Production Technologies 171 - Ani...,Nominal,Discrete
2,Daytime/evening attendance\t,1 - daytime 0 - evening,Nominal,Discrete
3,Previous qualification,1 - Secondary education 2 - Higher education -...,Nominal,Discrete
4,Previous qualification (grade),Grade of previous qualification (between 0 and...,Numeric,Continous
5,Nacionality,1 - Portuguese; 2 - German; 6 - Spanish; 11 - ...,Nominal,Discrete
6,Mother's qualification,1 - Secondary Education - 12th Year of Schooli...,Nominal,Discrete
7,Father's qualification,1 - Secondary Education - 12th Year of Schooli...,Nominal,Discrete
8,Mother's occupation,0 - Student 1 - Representatives of the Legisla...,Nominal,Discrete
9,Father's occupation,0 - Student 1 - Representatives of the Legisla...,Nominal,Discrete


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Marital status                       4424 non-null   int64  
 1   Course                               4424 non-null   int64  
 2   Daytime/evening attendance	          4424 non-null   int64  
 3   Previous qualification               4424 non-null   int64  
 4   Previous qualification (grade)       4424 non-null   float64
 5   Nacionality                          4424 non-null   int64  
 6   Mother's qualification               4424 non-null   int64  
 7   Father's qualification               4424 non-null   int64  
 8   Mother's occupation                  4424 non-null   int64  
 9   Father's occupation                  4424 non-null   int64  
 10  Admission grade                      4424 non-null   float64
 11  Displaced                     

After dropping all unwanted columns and one hot encoding the 'Student_Status' column, we are left with 26 features. Out the 26 features, there are 17 one-hot encoded features that's discrete and 9 features that are continuous. The next step we will take is to normalize the continuous features.

#### Scaling Numerical Features on our dataset.
First, we are selecting the continuous features from the dataset.


In [40]:
# Identify continuous features based on attribute descriptions
continuous_features = attr_des.loc[attr_des['Continous/Discrete'] == 'Continous', 'Features'].tolist()
print(f"Continuous Features: {continuous_features}")

Continuous Features: ['Previous qualification (grade)', 'Admission grade', 'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (grade)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (grade)', 'Unemployment rate', 'Inflation rate', 'GDP']


Then we are performing normalization with sklearn's preprocessing.normalize function. That does is scale each feature to have a unit norm. We can see that the continuous features are now normalized. Here is also where you are going to find the summary statistics for all the features in the dataset.

In [41]:
from sklearn import preprocessing
import numpy as np

df[continuous_features] = preprocessing.normalize(df[continuous_features])
df.info()
df.head()

# Select all numerical variables
num_columns = df.select_dtypes(float)


# Get summary statistics for numerical variables
summary_stats = num_columns.describe()
print(summary_stats)


# Categorical Variables Summary
df_temp = df.copy(deep=True)
df.info()


# Select all categorical variables
c = df_temp.select_dtypes(np.integer).columns


# Change all the ints to type of category
df_temp[c] = df_temp[c].astype('category')


# Get counts, unique values, 'top' value, and the frequency of each
cat_summary = df_temp.select_dtypes('category').describe()


print(cat_summary)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Marital status                       4424 non-null   int64  
 1   Course                               4424 non-null   int64  
 2   Daytime/evening attendance	          4424 non-null   int64  
 3   Previous qualification               4424 non-null   int64  
 4   Previous qualification (grade)       4424 non-null   float64
 5   Nacionality                          4424 non-null   int64  
 6   Mother's qualification               4424 non-null   int64  
 7   Father's qualification               4424 non-null   int64  
 8   Mother's occupation                  4424 non-null   int64  
 9   Father's occupation                  4424 non-null   int64  
 10  Admission grade                      4424 non-null   float64
 11  Displaced                     

#### Identify Cross-Product Features
We will now identify the cross-product features that we will use in our wide and deep network. A quick search on google scholar yield some interesting results. We will use the following features for our wide and deep network.

##### Curricular Units Enrolled × Curricular Units Grade
The classic excuse of "I'm taking too many classes" is often used by students to explain their poor grades. This 2021 paper on the role of academic workload in the performance of university students found that the number of curricular units enrolled is negatively correlated with the grade average. We will use this cross-product feature to show the interaction between the number of curricular units enrolled and the grade average.

https://eric.ed.gov/?id=EJ1304834

Reference: Schmid, C., Neuböck, S., & Horn, K. (2021). The role of academic workload in the performance of university students. Journal of Education and Work.

In [42]:
df['Curricular_Units_Interaction'] = df['Curricular units 1st sem (enrolled)'] * df['Curricular units 1st sem (grade)']

##### Previous Qualification × Grade of Previous Qualification
A 2005 study on the relationship between academic performance and previous qualifications found that the type and grade of the previous qualification is a significant predictor of academic performance. We will use this cross-product feature to show the interaction between the previous qualification and the grade of the previous qualification.

https://pubmed.ncbi.nlm.nih.gov/14717633/

Reference: Kuncel, N. R., Hezlett, S. A., & Ones, D. S. (2005). Academic performance, career potential, creativity, and job performance: Can one construct predict them all?. Journal of Personality and Social Psychology.

In [43]:
df['Qualification_Grade_Interaction'] = df['Previous qualification'] * df['Previous qualification (grade)']

##### Curricular Units (1st Semester) Grade × Scholarship Holder
A 2003 study on the effect of student aid on college attendance and completion found that scholarship holders have higher academic performance compared to non-scholarship holders. We will use this cross-product feature to show the interaction between the grade in the first semester and the scholarship holder status.

https://www.aeaweb.org/articles?id=10.1257/000282803321455287

Reference: Dynarski, S. (2003). Does aid matter? Measuring the effect of student aid on college attendance and completion. American Economic Review.



In [44]:
df['Scholarship_Performance_Interaction'] = df['Curricular units 1st sem (grade)'] * df['Scholarship holder']


##### GDP × Unemployment Rate
A 2012 study on the short- and long-term career effects of graduating in a recession found that macroeconomic factors like GDP and unemployment rates can jointly influence student decisions to enroll, persist, or drop out. We will use this cross-product feature to show the interaction between the GDP and the unemployment rate.

https://www.researchgate.net/publication/227349860_The_Short-_and_Long-Term_Career_Effects_of_Graduating_in_a_Recession

Reference: Oreopoulos, Philip, Till von Wachter, and Andrew Heisz. 2012. "The Short- and Long-Term Career Effects of Graduating in a Recession." American Economic Journal: Applied Economics, 4 (1): 1–29. DOI: 10.1257/app.4.1.1

In [45]:
df['Economic_Interaction'] = df['GDP'] * df['Unemployment rate']

To conclude, we added these 4 new cross product features to our dataset to be used in our wide and deep network. These features are the interaction between the number of curricular units enrolled and the grade average, the previous qualification and the grade of the previous qualification, the grade in the first semester and the scholarship holder status, and the GDP and the unemployment rate. This will put our dataset at 30 features.

In [46]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 30 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Marital status                       4424 non-null   int64  
 1   Course                               4424 non-null   int64  
 2   Daytime/evening attendance	          4424 non-null   int64  
 3   Previous qualification               4424 non-null   int64  
 4   Previous qualification (grade)       4424 non-null   float64
 5   Nacionality                          4424 non-null   int64  
 6   Mother's qualification               4424 non-null   int64  
 7   Father's qualification               4424 non-null   int64  
 8   Mother's occupation                  4424 non-null   int64  
 9   Father's occupation                  4424 non-null   int64  
 10  Admission grade                      4424 non-null   float64
 11  Displaced                     

,Marital status,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,...,Curricular units 2nd sem (enrolled),Curricular units 2nd sem (grade),Unemployment rate,Inflation rate,GDP,Student_Status,Curricular_Units_Interaction,Qualification_Grade_Interaction,Scholarship_Performance_Interaction,Economic_Interaction
0,1,171,1,1,0.690568,1,19,12,5,9,...,0.000000,0.000000,0.061132,0.007925,0.009849,0,0.000000,0.690568,0.0,0.000602
1,1,9254,1,1,0.741544,1,1,3,3,3,...,0.027808,0.063340,0.064422,-0.001390,0.003661,1,0.001804,0.741544,0.0,0.000236
2,1,9070,1,1,0.696828,1,37,37,9,9,...,0.034270,0.000000,0.061686,0.007996,0.009938,0,0.000000,0.696828,0.0,0.000613
3,1,9773,1,1,0.707999,1,38,37,5,3,...,0.034820,0.071961,0.054551,-0.004643,-0.018106,1,0.002713,0.707999,0.0,-0.000988
4,2,8014,0,1,0.571575,1,37,38,9,9,...,0.034295,0.074305,0.079449,-0.001715,0.004515,1,0.002418,0.571575,0.0,0.000359


#### Evaluation Metrics and Acceptence Criteria
